# Deep Q-Network

In [1]:
# Set relative path to parent directory
import sys, os
sys.path.insert(0, os.path.abspath('..'))

In [2]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import matplotlib.pyplot as plt
import numpy as np

checkpoint_file = "./weights/dqn.ckpt"

## The deep q network class

In [6]:

class Qnetwork():
    def __init__(self, learnig_rate, state_size, action_size, hidden_layer_shape, hidden_output_size):
        # Here we create the input of the network, and a hidden layer
        self.state_in = tf.placeholder(shape=[None,state_size],dtype=tf.float32)
        self.hidden = slim.stack(inputs = self.state_in,
                                 layer = slim.fully_connected, 
                                 stack_args = hidden_layer_shape, 
                                 biases_initializer = None, 
                                 activation_fn = tf.nn.relu)
        
        # We take the input state and split into a value and advangage sub network
        self.value_in = slim.flatten(self.hidden)
        self.advantage_in = slim.flatten(self.hidden)
        xavier_init = tf.contrib.layers.xavier_initializer()
        self.advantage_weights = tf.Variable(xavier_init([hidden_output_size,action_size]))
        self.value_weights = tf.Variable(xavier_init([hidden_output_size,1]))
        self.advantage = tf.matmul(self.advantage_in,self.action_weights)
        self.value = tf.matmul(self.value_in,self.value_weights)
        
        # We then combine the two sub netorks to get our final Q-values.
        self.Q_out = self.value + tf.subtract(self.Advantage,tf.reduce_mean(self.Advantage,axis=1,keep_dims=True))
        self.predict = tf.argmax(self.Q_out,1)
        
        # We define the loss as the sum of squares difference between the target and prediction Q values.
        self.target_Q = tf.placeholder(shape=[None],dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions,env.actions,dtype=tf.float32)
        
        self.Q = tf.reduce_sum(tf.multiply(self.Q_out, self.actions_onehot), axis=1)
        
        self.td_error = tf.square(self.targetQ - self.Q)
        self.loss = tf.reduce_mean(self.td_error)
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
        self.updateModel = self.trainer.minimize(self.loss)